# CartoonGAN : Generative Adversarial Networks for Photo Cartoonization
  

# Abstract(논문 개요)
- 사진을 카툰화해주는 생성적 적대적 신경망
- 분야 : computer vision, computer graphics
- 최근들어 이미지를 스타일링하는 것이 유행하고 있다.
- 하지만, 만족스러운 결과를 만들어내지는 못했다.
  - 이유 1. 만화는 높은 수준의 단순화 및 추상화와 함께 독특한 특징들을 가지고 있다.
  - 이유 2. 만화는 경계선이 깔끔하고, 색이 부드럽고, 상대적으로 단순한 텍스쳐를 가지는 경향이 있다. 따라서, 기존 방식인 텍스쳐를 기반으로 한 손실함수로는 상당한 어려움이 존재한다.
- CartoonGAN은 만화 형식으로 스타일링해주는 GAN 모델이다.
- 훈련 데이터 : 짝지어 지지 않은 사진, 만화 이미지
- loss
  - 1. semantic content loss(의미론적 콘텐츠 손실)
  - 사진과 만화 사이의 상당한 스타일 변화를 다루기 위해 VGG 네트워크의 고수준의 feature map에서 희귀성을 위한 정규화를 만들어낸다.
  - 2. 깔끔한 경계선을 지키기 위한 경계선을 강조하는 적대적 loss
- 초기화 단계, 타켓 다양성(target manifold)를 위한 컨볼루젼 네트워크를 향상시키는 방법 소개


# 1. Introduction
- 만화는 일상 생활에서 널리 활용되고 있다.
- 많은 유명한 만화 이미지는 실제 장면을 기반으로 만들어졌다.
- 실제 장면을 만화 스타일로 수작업하는 것은 매우 힘들고 상당한 예술적 스킬을 포함한다.
- 하지만, 현재 나와있는 소프트웨어나 알고리즘으로는 만족할만한 결과물이 나오지 않는다.
- 따라서 자동으로 실제 장면을 카툰화해주는 것이 생기면 아티스트에게도 도움이 되고, 시간 절약도 되고, 좀 더 창조적인 일에 집중할 수 있게 된다.
- 또한, 인스타그램, 포토샵과 같은 사진 편집 소프트웨어에도 유용한 도움이 된다.
- 예술적 방식으로 이미지를 스타일링 하는 방식은 비사실적 렌더링 분야에서 광범위하게 연구되었다.
- 기존의 방식은 특정 스타일에 대한 전용 알고리즘을 개발한다.
- 하지만, 각각의 아티스트를 모방하는 것은 상당한 노력이 필요하다.
- 최근에는 주어진 이미지를 기반으로 스타일링이 가능한 학습 기반 스타일 변화 방법이 상당한 주목을 받고 있다.
- 특히, 짝지어지지 않은 사진과 스타일된 이미지로부터 훈련된다는 독특한 특징을 가지고 있는 GAN 모델이.
- 학습 기반 스타일 변화 방법은 상당한 성공을 거두었지만 괜찮은 품질의 카툰화된 이미지를 만들어내는 것에는 실패했다.
  - 이유 1. 텍스쳐를 추가하는 것 대신에, 만화는 실제 모습과 비교하여 매우 단순하고 추상적이다.
  - 이유 2. 아티스트들의 다양한 스타일에도 불구하고, 만화는 다른 형태의 예술 작품과는 다르게 눈에띄는 공통적인 특성을 갖는다. - 경계선이 깔끔하고, 부드러운 색상 음영, 상대적으로 단순한 텍스쳐를 갖는다.
- 훈련 데이터 : 사진 데이터셋, 만화 이미지 데이터셋
- 훈련 데이터로부터 고 품질의 결과를 만들어 내기 쉽게 하기 위해서, 사진과 만화 두 데이터셋에서 일치하거나 짝지어지는 데이터를 필요로 하지 않는다.
- computer vision 알고리즘 관점으로부터, 만화 스타일링의 목표는 컨텐츠를 변경하지 않는 동안 여러 개의 사진을 여러 개의 만화로 매핑하는 것이다.
- 이 목표를 달성하기 위해서, 우리는 전용 GAN 기반 architecture를 사용할 것을 제안한다. 간단하면서도 효과적인 손실 함수 2개를 사용하면서.
- 이 논문의 중요한 핵심 2가지
  - 1. 짝지어지지 않은 훈련을 위한 사진, 만화 데이터셋 사용.
  - 최신 기법보다 더 좋은 품질의 카툰화 이미지 출력이 목표.
  - 개개인의 아티스트의 만화를 훈련에 사용하면, 우리의 도구는 그들의 스타일로 재생산해준다.
  - 2. GAN기반의 architecture에서 간단하지만 효과적인 2개의 손실 함수를 이용.
  - 생성 신경망에서, 사진과 만화 사이의 상당한 스타일 차이를 다루기 위해서 우리는 의미론적 손실을 VGG 네트워크의 고수준 feature map에서의 $l_{1}$ 희귀성 정규화(sparse reguralization)로 소개하겠다.


# 추가 개념 설명
## 비사실적 렌더링
- 컴퓨터 그래픽스의 한 영역. 사실적인 렌더링 이외의 표현. 
- 양식. 회화, 드로잉, 만화 같이 렌더링하는 것.

## 희소 표현(sparse representation)
- 대부분 0으로 표현되는 것. 종종 차원의 크기가 크다.

## 특성 교차
- 특성들을 조합하여 새로운 특성을 만들어내는 것
- 원래보다 차원의 크기가 커진다.
- 원래보다 자원을 더 많이 필요로 한다.

## 자원을 많이 차지할 때
- 가중치가 정확하게 0으로 떨어지도록 유도하는 것이 좋다.
- 가중치가 정확하게 0이 될 경우 모델에서 해당 특성을 삭제하게 되므로, 자원 소모량이 줄어들게 된다.

## 희소성을 위한 정규화 : $l_{1}, l_{2}$ 규제(정규화)
- 규제 : 가중치의 성장을 제한한다. 일반적인 패턴이 아닌 몇몇 독특하면서 희소한 패턴을 가지는 데이터(noise, outlier)로부터 영향을 적게 받겠다.
